In [2]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version: ', tf.__version__)
print('Eager mode: ', tf.executing_eagerly())
print('Hub version: ', hub.__version__)
print('GPU is available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

Version:  2.6.0
Eager mode:  True
Hub version:  0.12.0
GPU is available


In [3]:
train_data, validation_data, test_data = tfds.load(name = 'imdb_reviews', split=('train[:60%]', 'train[60%:]', 'test'), as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePMD1VV/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompletePMD1VV/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

# 0 : 부정적 리뷰 1: 긍정적 리뷰

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
# tensorflow hub에서 사전 훈련된 텍스트 임베팅 모델 사용

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer) # 사전 훈련된 저장된 모델을 사용하여 문장을 임베팅 벡터에 매핑
model.add(tf.keras.layers.Dense(16, activation = 'relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer = 'adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics = ['accuracy'])

In [11]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs = 10, validation_data = validation_data.batch(512), verbose = 1)

Epoch 1/10
30/30 [==============================] - 7s 110ms/step - loss: 0.6561 - accuracy: 0.5277 - val_loss: 0.5918 - val_accuracy: 0.5776
Epoch 2/10
30/30 [==============================] - 4s 101ms/step - loss: 0.5256 - accuracy: 0.6888 - val_loss: 0.4873 - val_accuracy: 0.7222
Epoch 3/10
30/30 [==============================] - 4s 107ms/step - loss: 0.4017 - accuracy: 0.8172 - val_loss: 0.3985 - val_accuracy: 0.8142
Epoch 4/10
30/30 [==============================] - 4s 100ms/step - loss: 0.2983 - accuracy: 0.8883 - val_loss: 0.3456 - val_accuracy: 0.8435
Epoch 5/10
30/30 [==============================] - 4s 102ms/step - loss: 0.2229 - accuracy: 0.9215 - val_loss: 0.3185 - val_accuracy: 0.8652
Epoch 6/10
30/30 [==============================] - 4s 102ms/step - loss: 0.1674 - accuracy: 0.9469 - val_loss: 0.3056 - val_accuracy: 0.8699
Epoch 7/10
30/30 [==============================] - 4s 104ms/step - loss: 0.1255 - accuracy: 0.9647 - val_loss: 0.3020 - val_accuracy: 0.8717
Epoch 

In [12]:
results = model.evaluate(test_data.batch(512), verbose = 2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f " % (name, value))

49/49 - 2s - loss: 0.3473 - accuracy: 0.8510
loss: 0.347 
accuracy: 0.851 
